# 102 Category Flower Dataset

- pytorch dataset(https://docs.pytorch.org/vision/main/datasets.html)
    - FashionMNIST을 사용해서 최소한의 모델을 구성해봅시다!
    - caltech101

- Models and pretrained weights
    - resnet152(pytorch:: https://docs.pytorch.org/vision/0.20/models/resnet.html)

CNN:: 
- tensor -> conv(숨겨진 특성추출)
- 이미지에 국한되지 마라

In [166]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets, transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'GPU.CPU: {device}')

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

GPU.CPU: cpu


In [167]:
batch_size = 64
lr = 0.001
num_epochs = 20
num_classes = 102

## 1. 전처리(이미지)

- torchvision -> openCV

In [168]:
# 1. 이미지를 32, 32로 사이즈 변경해야 함
# 2. 텐서로 변경해야 함❗
train_transforms = transforms.Compose([
    # _____________________1,
    transforms.Resize((32,32)),
    transforms.RandomHorizontalFlip(p=0.5), # 데이터 증감 # 이미지특성을 더 발현시킴
    # _____________________2,
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)) # ImageNet 정규화:: 수많은 수행을 통해 알아낸 정규화 잘되는 숫자
])
val_transforms = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor()
    ])


## 2. 데이터 불러오기

In [169]:
train_dataset = datasets.Flowers102(root='./data', split='train', transform=train_transforms, download=True) # dataset받아뒀을 때 False를 하는데 다를 수 있어서 True하세요
val_dataset = datasets.Flowers102(root='./data', split='val', transform=val_transforms, download=True)
test_dataset = datasets.Flowers102(root='./data', split='test', transform=val_transforms, download=True)

In [170]:
# datasets.FashionMNIST(root='./data', download=True)

In [171]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)   # batch_size 안 정하면 한번에 다 들고 오니까 터질 수 있음
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [172]:
# 검증데이터 개수 알아보기
print(f'훈련 데이터: {len(train_loader)}')
print(f'검증 데이터: {len(val_loader)}')
print(f'테스트 데이터: {len(test_loader)}')

훈련 데이터: 16
검증 데이터: 16
테스트 데이터: 97


In [180]:
data_iter = iter(train_loader)
sample_batch = next(data_iter)
sample_image, sample_label = sample_batch
sample_image.shape

torch.Size([64, 3, 32, 32])

## 3. 모델 설계

- 전이학습이 되는 이유
- input숫자와 output숫자 맞추기

### 3-1. LeNet5

In [174]:
# class LeNet5Classic(nn.Module): # 옛날꺼__성능:(
#     def __init__(self, *args, **kwargs):
#         super(LeNet5Classic, self).__init__()
#         # ❗conv1 -> conv2 -> conv3 -> fc1 -> fc2
#         self.conv1 = nn.Conv2d(3, 6, kernel_size=5)     # 커널사이즈를 기반으로 특징 추출   # 숫자(하이퍼파라미터)는 성능에 크게 영향을 줌
#         self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
#         self.conv3 = nn.Conv2d(16, 120, kernel_size=5)
#         self.fc1 = nn.Linear(120, 84)                   # fully connected:: 분류기
#         self.fc2 = nn.Linear(84, num_classes)

#         self.relu = nn.ReLU()
#         self.pool = nn.AvgPool2d(kernel_size=2, stride=2) # MaxPool과 달리 사이즈가 안바뀜

#     def forward(self, x):
#         x = self.pool(self.relu(self.conv1(x))) # 32 * 32 * 3 -> 28 * 28 * 6 -> 14 * 14 * 6
#         x = self.pool(self.relu(self.conv2(x))) # 14 * 14 * 6 -> 10 * 10 * 16 -> 5 * 5 * 16
#         x = self.relu(self.conv3(x))            # 5 * 5 * 16 -> 1 * 1 *120
#         x = torch.flatten(x,1)                    # 120   # ❗
#         x = self.relu(self.fc1(x))              # 120 -> 84
#         x = self.fc2(x)                         # 84 -> num_classes
#         return x

In [175]:
class LeNet5Modern(nn.Module):  # 요즘꺼
    def __init__(self, *args, **kwargs):
        super(LeNet5Modern, self).__init__()
        # 특성 추출
        self.features = nn.Sequential(
            # 첫번째 합성곱 레이어: 3 -> 6 채널
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # 두번째 합성곱 레이어: 6 -> 16 채널
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # 세번째 합성곱 레이어: 16 -> 120 채널 (:: 한이미지의 특징이 120개)
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(120),
            nn.ReLU(inplace=True),
        )

        # 분류
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(120, 84),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(84, num_classes)
        )

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan-out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)    # 강제로 bias값 0으로 바꿈
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


### 3-2. 모델 설계(수업에 사용된 모델) 학습

In [ ]:
class Flower102CNN(nn.Module): # 변경해보세요
    def __init__(self):
        super(Flower102CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1), # 사이즈:: 32 * 32 * 3(채널이 3개니까 필터값이 3*3*3) => 32 * 32 * 32   ## Out = (입력크기 + 2*패딩 - 커널크기) / 스트라이드 + 1
            nn.BatchNorm2d(32),                         # 배치정규화 => 32개가 나감
            nn.ReLU(),                                  # 32
            nn.MaxPool2d(kernel_size=2, stride=2)       # 16 * 16 * 32                          ## 32*32 >> 16*16 == 입력된 이미지 값이 작아지는거
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3),   #작은걸 확대시키니까 특성이 더 발현됨
            nn.BatchNorm2d(64),
            nn.ReLU(),                          # 14 * 14 * 64(padding을 안해서 14가 나옴)
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc1 = nn.Linear(64 * 7 * 7, 512) 
        self.drop = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, num_classes)  # 출력개수: 102

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)   # 옛날 코드엔 flatten코드가 없어서
        x = self.fc1(x)
        x = self.drop(x)
        x = self.fc2(x)
        return x

## 4. 학습

In [177]:
# 3. 손실 함수 및 옵티마이저 정의
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. 학습 루프
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # 순전파
            outputs = model(images)
            loss = criterion(outputs, labels) #

            # 역전파 및 최적화
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")


# model = Flower102CNN()
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)

# 5. 학습 및 평가
# model = LeNet5Classic().to(device)
model = LeNet5Modern().to(device)
# model = Flower102CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model, train_loader, criterion, optimizer, num_epochs=20) # epochs 몇번돌릴건지

Epoch [1/20], Loss: 4.6367
Epoch [2/20], Loss: 4.5398
Epoch [3/20], Loss: 4.4663
Epoch [4/20], Loss: 4.3766
Epoch [5/20], Loss: 4.2418
Epoch [6/20], Loss: 4.0739
Epoch [7/20], Loss: 3.9552
Epoch [8/20], Loss: 3.8367
Epoch [9/20], Loss: 3.7242
Epoch [10/20], Loss: 3.6023
Epoch [11/20], Loss: 3.5084
Epoch [12/20], Loss: 3.4340
Epoch [13/20], Loss: 3.3149
Epoch [14/20], Loss: 3.2682
Epoch [15/20], Loss: 3.1742
Epoch [16/20], Loss: 3.1206
Epoch [17/20], Loss: 3.0702
Epoch [18/20], Loss: 2.9886
Epoch [19/20], Loss: 2.9353
Epoch [20/20], Loss: 2.8639


## 5. 평가

In [179]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0 # ❗
    total = 0   # ❗
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
evaluate_model(model, test_loader)

Test Accuracy: 1.17%
